In [8]:
import torch

from systems import SO, LienardSigmoid, LienardPoly, Selkov, BZreaction, VanDerPol
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [3]:
N = 1000 # number of points
T_max = 3 # maximum simulation time
n_systems = 200 # number of systems to create
dim = 2 # dimension of data
root = 'data/'

systems = [
    ['SO', SO],
    ['selkov', Selkov],
    ['bzreaction', BZreaction],
    ['lienard_poly', LienardPoly],
    ['lienard_sigmoid', LienardSigmoid],
    ['vanderpol', VanDerPol],
]

In [ ]:
for (name, system) in systems:
    path = root + f'{name}/'
    Path(path).mkdir(exist_ok=True, parents=True)
    xs = []
    dxs = []
    print(name, flush=True)
    for i in tqdm(range(n_systems)):
        torch.manual_seed(i)
        np.random.seed(i)
        s = system()
        x = s.rand_on_traj(s.random_x(N=N, dim=dim), T=T_max)
        xs.append(x.numpy())
        dx = s(0, x)
        dxs.append(dx.numpy())
    xs = np.stack(xs)
    np.save(path+f'coords_T={T_max}.npy', xs)
    dxs = np.stack(dxs)
    np.save(path+f'X_T={T_max}.npy', dxs)